# 🔹UFC Fight Predictor ETL

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Import Libraries and Setup Environment

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root))
from src.helpers import *

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Load Data

In [2]:
# Define the path to the CSV file
file_path = os.path.join(project_root, 'data', 'raw', 'ufc_raw.csv')

# Load the CSV into a DataFrame
try:
    ufc_raw = pd.read_csv(file_path)
    logger.info(f"✅ Data successfully loaded: {ufc_raw.shape[0]} rows, {ufc_raw.shape[1]} columns.")
except Exception as e:
    logger.error(f"❌ Error loading training data: {e}")

[INFO] ✅ Data successfully loaded: 6541 rows, 118 columns.


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Preview

In [3]:
# Preview the first few records
display(ufc_raw.head())

# General dataset information
ufc_raw.info()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Colby Covington,Joaquin Buckley,205.0,-250.0,205.0000,40.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,NaN,3.0,4:42,882.0,300.0,175.0,1800.0,2000.0,1100.0,150.0
1,Cub Swanson,Billy Quarantillo,124.0,-148.0,124.0000,67.5676,2024-12-14,"Tampa, Florida, USA",USA,Red,...,Punch,3.0,1:36,696.0,250.0,NaN,1800.0,NaN,450.0,NaN
2,Manel Kape,Bruno Silva,-395.0,310.0,25.3165,310.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,Punches,3.0,1:57,717.0,-105.0,550.0,900.0,1800.0,225.0,1100.0
3,Vitor Petrino,Dustin Jacoby,-340.0,270.0,29.4118,270.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,Punch,3.0,3:44,824.0,240.0,500.0,550.0,3000.0,110.0,800.0
4,Adrian Yanez,Daniel Marcos,185.0,-225.0,185.0000,44.4444,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,NaN,3.0,5:00,900.0,450.0,150.0,2200.0,2200.0,450.0,200.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6541 entries, 0 to 6540
Columns: 118 entries, RedFighter to BKOOdds
dtypes: bool(1), float64(60), int64(43), object(14)
memory usage: 5.8+ MB


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Check 

In [4]:
# Null values check
nulls = ufc_raw.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])

# Duplicate analysis
duplicates = ufc_raw.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

Null values per column:
 RedOdds                   227
BlueOdds                  226
RedExpectedValue          227
BlueExpectedValue         226
BlueAvgSigStrLanded       930
BlueAvgSigStrPct          765
BlueAvgSubAtt             832
BlueAvgTDLanded           833
BlueAvgTDPct              842
BlueStance                  3
RedAvgSigStrLanded        455
RedAvgSigStrPct           357
RedAvgSubAtt              357
RedAvgTDLanded            357
RedAvgTDPct               367
EmptyArena               1499
BMatchWCRank             5339
RMatchWCRank             4760
RWFlyweightRank          6445
RWFeatherweightRank      6532
RWStrawweightRank        6395
RWBantamweightRank       6387
RHeavyweightRank         6355
RLightHeavyweightRank    6357
RMiddleweightRank        6359
RWelterweightRank        6349
RLightweightRank         6357
RFeatherweightRank       6364
RBantamweightRank        6360
RFlyweightRank           6352
RPFPRank                 6288
BWFlyweightRank          6468
BWFeatherweightRank      6540
BWStrawweightRank        6441
BWBantamweightRank       6434
BHeavyweightRank         6393
BLightHeavyweightRank    6421
BMiddleweightRank        6404
BWelterweightRank        6421
BLightweightRank         6421
BFeatherweightRank       6417
BBantamweightRank        6422
BFlyweightRank           6410
BPFPRank                 6474
Finish                    238
FinishDetails            3643
FinishRound               622
FinishRoundTime           622
TotalFightTimeSecs        622
RedDecOdds               1087
BlueDecOdds              1117
RSubOdds                 1336
BSubOdds                 1360
RKOOdds                  1334
BKOOdds                  1361
dtype: int64

Duplicate rows: 0

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Data Cleaning

## Null Values

In [5]:
# Drop columns with too many null values (threshold: 1000)
ufc_deploy = ufc_raw.copy()
threshold = 1000
cols_to_drop = [col for col in ufc_deploy.columns if ufc_deploy[col].isnull().sum() > threshold]
for col in cols_to_drop:
    print('Dropping:', col)
ufc_deploy.drop(columns=cols_to_drop, inplace=True)

# Drop rows with any remaining missing values
print(f"➡️ Before dropna: {ufc_deploy.shape}")
ufc_deploy.dropna(inplace=True)
print(f"✅ After dropna: {ufc_deploy.shape}")

Dropping: EmptyArena

Dropping: BMatchWCRank

Dropping: RMatchWCRank

Dropping: RWFlyweightRank

Dropping: RWFeatherweightRank

...

Dropping: BSubOdds

Dropping: RKOOdds

Dropping: BKOOdds

➡️ Before dropna: (6541, 82)

✅ After dropna: (4719, 82)

In [6]:
# Drop columns with too many null values (threshold: 300)
threshold = 300
cols_to_drop = [col for col in ufc_raw.columns if ufc_raw[col].isnull().sum() > threshold]
for col in cols_to_drop:
    print('Dropping:', col)
ufc_raw.drop(columns=cols_to_drop, inplace=True)

# Drop rows with any remaining missing values
print(f"➡️ Before dropna: {ufc_raw.shape}")
ufc_raw.dropna(inplace=True)
print(f"✅ After dropna: {ufc_raw.shape}")

Dropping: BlueAvgSigStrLanded

Dropping: BlueAvgSigStrPct

Dropping: BlueAvgSubAtt

Dropping: BlueAvgTDLanded

Dropping: BlueAvgTDPct

...

Dropping: BSubOdds

Dropping: RKOOdds

Dropping: BKOOdds

➡️ Before dropna: (6541, 69)

✅ After dropna: (6062, 69)

In [7]:
# Null values check
nulls = ufc_raw.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])

Null values per column:
 Series([], dtype: int64)

In [8]:
ufc_raw.columns

Index(['RedFighter', 'BlueFighter', 'RedOdds', 'BlueOdds', 'RedExpectedValue',
       'BlueExpectedValue', 'Date', 'Location', 'Country', 'Winner',
       'TitleBout', 'WeightClass', 'Gender', 'NumberOfRounds',
       'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws',
       'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought',
       'BlueTotalTitleBouts', 'BlueWinsByDecisionMajority',
       'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous',
       'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage',
       'BlueWins', 'BlueStance', 'BlueHeightCms', 'BlueReachCms',
       'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak',
       'RedDraws', 'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought',
       'RedTotalTitleBouts', 'RedWinsByDecisionMajority',
       'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO',
       'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins',
       'RedStance', 'Red

## Incongruent Data, KNN Imputer

In [9]:
print(ufc_raw[['RedReachCms', 'BlueReachCms', 'RedHeightCms', 'BlueHeightCms']].describe())

RedReachCms  BlueReachCms  RedHeightCms  BlueHeightCms
count  6062.000000   6062.000000   6062.000000    6062.000000
mean    182.542097    182.342446    177.883032     177.901699
std      11.060124     11.139982      9.125898       9.068249
min     147.320000      0.000000    152.400000     152.400000
25%     175.260000    175.260000    170.180000     170.180000
50%     182.880000    182.880000    177.800000     177.800000
75%     190.500000    190.500000    185.420000     185.420000
max     214.630000    213.360000    210.820000     210.820000

In [10]:
from sklearn.impute import KNNImputer

# 1️⃣ Replace zeros with np.nan
ufc_raw['RedReachCms'] = ufc_raw['RedReachCms'].replace(0, np.nan)
ufc_raw['BlueReachCms'] = ufc_raw['BlueReachCms'].replace(0, np.nan)

# 2️⃣ Define relevant columns
cols = [
    'BlueReachCms', 'BlueHeightCms', 'BlueWeightLbs', 'BlueAge',
    'RedReachCms', 'RedHeightCms', 'RedWeightLbs', 'RedAge'
]

# 3️⃣ Ensure all missing values are np.nan (replace pd.NA, NaT, etc.)
ufc_raw[cols] = ufc_raw[cols].apply(pd.to_numeric, errors='coerce')

# 4️⃣ Apply KNNImputer
imputer = KNNImputer(n_neighbors=5)
ufc_raw[cols] = imputer.fit_transform(ufc_raw[cols])

In [11]:
print(ufc_raw[['RedReachCms', 'BlueReachCms', 'RedHeightCms', 'BlueHeightCms']].describe())

RedReachCms  BlueReachCms  RedHeightCms  BlueHeightCms
count  6062.000000   6062.000000   6062.000000    6062.000000
mean    182.542097    182.371190    177.883032     177.901699
std      11.060124     10.891442      9.125898       9.068249
min     147.320000    147.320000    152.400000     152.400000
25%     175.260000    175.260000    170.180000     170.180000
50%     182.880000    182.880000    177.800000     177.800000
75%     190.500000    190.500000    185.420000     185.420000
max     214.630000    213.360000    210.820000     210.820000

In [12]:
# Open Stance is incorrect
ufc_raw[ufc_raw['RedStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish
6051,Krzysztof Soszynski,Igor Pokrajac,-255.0,235.0,39.2157,235.0000,2011-12-10,"Toronto, Ontario, Canada",Canada,Blue,...,-1,-1,-2.54,-2.54,2,-15.5000,-0.5500,0.6250,neither,KO/TKO
6366,Krzysztof Soszynski,Goran Reljic,-108.0,-102.0,92.5926,98.0392,2010-11-13,"Oberhausen, North Rhine-Westphalia, Germany",Germany,Red,...,-1,-2,5.08,10.16,7,-4.5000,0.8333,0.8333,neither,U-DEC
6448,Krzysztof Soszynski,Stephan Bonnar,-190.0,175.0,52.6316,175.0000,2010-07-03,"Las Vegas, Nevada, USA",USA,Blue,...,-1,0,7.62,2.54,-1,3.2000,0.4000,0.1273,neither,KO/TKO
6511,Nate Quarry,Jorge Rivera,-230.0,190.0,43.4783,190.0000,2010-03-31,"Charlotte, North Carolina, USA",USA,Blue,...,-2,0,2.54,2.54,0,-2.8182,0.0909,0.4343,neither,KO/TKO


In [13]:
ufc_raw = ufc_raw[ufc_raw['RedStance'] != 'Open Stance']

In [14]:
# Open Stance is incorrect
ufc_raw[ufc_raw['RedStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish


In [15]:
# Open Stance is incorrect
ufc_raw[ufc_raw['BlueStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish
6216,Mike Massenzio,Krzysztof Soszynski,265.0,-325.0,265.0,30.7692,2011-06-11,"Vancouver, British Columbia, Canada",Canada,Blue,...,2,1,-2.54,10.16,-5,28.4762,-0.8095,-1.5714,neither,U-DEC


In [16]:
ufc_raw = ufc_raw[ufc_raw['BlueStance'] != 'Open Stance']

In [17]:
# Open Stance is incorrect
ufc_raw[ufc_raw['BlueStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish


In [18]:
# Open Stance is incorrect
ufc_raw[ufc_raw['RedStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish


## Create Fight Stance Columns
- If both fighters have the same fighting stance, the bout is considered a Closed Stance matchup. If their stances differ, it is classified as an Open Stance matchup.

In [19]:
# Create column FightStance according Stances matches.
ufc_raw['FightStance'] = np.where(
    ufc_raw['BlueStance'] == ufc_raw['RedStance'],
    'Closed Stance',
    'Open Stance'
)

In [20]:
ufc_preview = ufc_raw[ufc_raw['FightStance'] == 'Open Stance']

In [21]:
ufc_preview2 = ufc_raw[ufc_raw['FightStance'] == 'Closed Stance']

In [22]:
ufc_preview[['FightStance', 'BlueStance', 'RedStance']]

,FightStance,BlueStance,RedStance
0,Open Stance,Southpaw,Orthodox
2,Open Stance,Orthodox,Southpaw
6,Open Stance,Switch,Southpaw
10,Open Stance,Orthodox,Southpaw
11,Open Stance,Southpaw,Orthodox
...,...,...,...
6524,Open Stance,Orthodox,Southpaw
6525,Open Stance,Orthodox,Southpaw
6528,Open Stance,Orthodox,Switch
6529,Open Stance,Orthodox,Southpaw


In [23]:
ufc_preview2[['FightStance', 'BlueStance', 'RedStance']]

,FightStance,BlueStance,RedStance
1,Closed Stance,Orthodox,Orthodox
3,Closed Stance,Orthodox,Orthodox
4,Closed Stance,Orthodox,Orthodox
5,Closed Stance,Orthodox,Orthodox
7,Closed Stance,Orthodox,Orthodox
...,...,...,...
6535,Closed Stance,Orthodox,Orthodox
6536,Closed Stance,Orthodox,Orthodox
6538,Closed Stance,Orthodox,Orthodox
6539,Closed Stance,Orthodox,Orthodox


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Check Clean Data

In [24]:
# Null values check
nulls = ufc_raw.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])

# Duplicate analysis
duplicates = ufc_raw.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

Null values per column:
 Series([], dtype: int64)

Duplicate rows: 0

In [25]:
# Preview the first few records
display(ufc_raw.head())
display(ufc_raw.columns)
# Para ver los tipos de dato de cada columna:
display(ufc_raw.dtypes)

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish,FightStance
0,Colby Covington,Joaquin Buckley,205.0,-250.0,205.0000,40.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,-2,-2.54,10.16,-6,0.25,-0.2,-1.83,Red,KO/TKO,Open Stance
1,Cub Swanson,Billy Quarantillo,124.0,-148.0,124.0000,67.5676,2024-12-14,"Tampa, Florida, USA",USA,Red,...,-1,5.08,0.00,-5,2.69,0.7,0.20,neither,KO/TKO,Closed Stance
2,Manel Kape,Bruno Silva,-395.0,310.0,25.3165,310.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,1,-2.54,-7.62,3,-1.12,-0.2,1.72,Red,KO/TKO,Open Stance
3,Vitor Petrino,Dustin Jacoby,-340.0,270.0,29.4118,270.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,-1,2.54,-2.54,9,2.68,-0.8,-3.62,neither,KO/TKO,Closed Stance
4,Adrian Yanez,Daniel Marcos,185.0,-225.0,185.0000,44.4444,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,0,0.00,-2.54,0,-0.57,0.0,0.25,neither,S-DEC,Closed Stance


Index(['RedFighter', 'BlueFighter', 'RedOdds', 'BlueOdds', 'RedExpectedValue',
       'BlueExpectedValue', 'Date', 'Location', 'Country', 'Winner',
       'TitleBout', 'WeightClass', 'Gender', 'NumberOfRounds',
       'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws',
       'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought',
       'BlueTotalTitleBouts', 'BlueWinsByDecisionMajority',
       'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous',
       'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage',
       'BlueWins', 'BlueStance', 'BlueHeightCms', 'BlueReachCms',
       'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak',
       'RedDraws', 'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought',
       'RedTotalTitleBouts', 'RedWinsByDecisionMajority',
       'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO',
       'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins',
       'RedStance', 'Red

RedFighter           object
BlueFighter          object
RedOdds             float64
BlueOdds            float64
RedExpectedValue    float64
                     ...   
AvgSubAttDif        float64
AvgTDDif            float64
BetterRank           object
Finish               object
FightStance          object
Length: 70, dtype: object

# Create the target value: **0** (Fighter Red wins) or **1** (Fighter Blue wins)

In [26]:
ufc_raw['label'] = ufc_raw['Winner'].apply(lambda x: 1 if x == 'Blue' else 0)
ufc_raw=ufc_raw.drop('Winner', axis=1)

ufc_deploy['label'] = ufc_deploy['Winner'].apply(lambda x: 1 if x == 'Blue' else 0)
ufc_deploy=ufc_deploy.drop('Winner', axis=1)

# Save

In [27]:
# Save the cleaned file
ufc_raw.to_csv(f'{project_root}/data/processed/ufc_etl.csv', index=False)
logger.info("✅ ETL file saved as 'ufc_etl.csv'.")

[INFO] ✅ ETL file saved as 'ufc_etl.csv'.


In [28]:
# Save the cleaned file
ufc_deploy.to_csv(f'{project_root}/data/processed/ufc_deploy.csv', index=False)
logger.info("✅ Deploy file saved as 'ufc_deploy.csv'.")

[INFO] ✅ Deploy file saved as 'ufc_deploy.csv'.


# Create column from Dates and Fighter names for the Dataset

# Load Data

In [29]:
# Define the path to the CSV file
file_path = os.path.join(project_root, 'data', 'raw', 'ufc_raw.csv')

# Load the CSV into a DataFrame
try:
    ufc_raw = pd.read_csv(file_path)
    logger.info(f"✅ Data successfully loaded: {ufc_raw.shape[0]} rows, {ufc_raw.shape[1]} columns.")
except Exception as e:
    logger.error(f"❌ Error loading training data: {e}")

[INFO] ✅ Data successfully loaded: 6541 rows, 118 columns.


## Transform Date to datetime64

In [30]:
ufc_raw['Date']

0       2024-12-14
1       2024-12-14
2       2024-12-14
3       2024-12-14
4       2024-12-14
           ...    
6536    2010-03-21
6537    2010-03-21
6538    2010-03-21
6539    2010-03-21
6540    2010-03-21
Name: Date, Length: 6541, dtype: object

In [31]:
ufc_raw['Date'] = pd.to_datetime(ufc_raw['Date'], errors='coerce')

In [32]:
n_invalid = ufc_raw['Date'].isna().sum()
print(f" ✅ Conversion completed. Invalid dates: {n_invalid}")

✅ Conversion completed. Invalid dates: 0

In [33]:
ufc_raw['Date']

0      2024-12-14
1      2024-12-14
2      2024-12-14
3      2024-12-14
4      2024-12-14
          ...    
6536   2010-03-21
6537   2010-03-21
6538   2010-03-21
6539   2010-03-21
6540   2010-03-21
Name: Date, Length: 6541, dtype: datetime64[ns]

In [34]:
# Ordenar por fecha ascendente
ufc_raw = ufc_raw.sort_values(by='Date').reset_index(drop=True)

In [35]:
ufc_raw['Date']

0      2010-03-21
1      2010-03-21
2      2010-03-21
3      2010-03-21
4      2010-03-21
          ...    
6536   2024-12-14
6537   2024-12-14
6538   2024-12-14
6539   2024-12-14
6540   2024-12-14
Name: Date, Length: 6541, dtype: datetime64[ns]

In [36]:
ufc_raw['RedFighter'] = ufc_raw['RedFighter'].str.strip()
ufc_raw['BlueFighter'] = ufc_raw['BlueFighter'].str.strip()
ufc_raw

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Eric Schafer,Jason Brilz,140.0,-160.0,140.0000,62.5000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Brandon Vera,Jon Jones,215.0,-235.0,215.0000,42.5532,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,Elbow,1.0,3:19,199.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Junior Dos Santos,Gabriel Gonzaga,-250.0,230.0,40.0000,230.0000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,Punches,1.0,3:53,233.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Cheick Kongo,Paul Buentello,-345.0,315.0,28.9855,315.0000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,Elbows,3.0,1:16,676.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Alessio Sakara,James Irvin,-120.0,100.0,83.3333,100.0000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,1.0,3:01,181.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6536,Miranda Maverick,Jamey-Lyn Horth,-625.0,455.0,16.0000,455.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,NaN,3.0,5:00,900.0,-295.0,650.0,500.0,2500.0,1400.0,2200.0
6537,Davey Grant,Ramon Taveras,-122.0,102.0,81.9672,102.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,NaN,3.0,5:00,900.0,225.0,350.0,750.0,800.0,350.0,300.0
6538,Josefine Knutsson,Piera Rodriguez,-245.0,200.0,40.8163,200.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,NaN,3.0,5:00,900.0,-175.0,300.0,1800.0,1400.0,800.0,1200.0
6539,Michael Johnson,Ottman Azaitar,-230.0,190.0,43.4783,190.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,Punch,2.0,2:03,423.0,300.0,650.0,1000.0,2500.0,120.0,350.0


## Create DaysSinceLastFight: Days since his last UFC fight

In [37]:
# Sort the DataFrame by ascending fight date
ufc_raw = ufc_raw.sort_values(by='Date').reset_index(drop=True)

# Dictionary to store each fighter's last fight date
last_fight_date_by_fighter = {}

# Lists to store the computed days since last fight
red_days_since_last_fight = []
blue_days_since_last_fight = []

# Iterate through each fight record
for idx, row in ufc_raw.iterrows():
    current_date = row['Date']
    red_fighter = row['RedFighter']
    blue_fighter = row['BlueFighter']

    # Days since last fight for Red corner
    red_last_date = last_fight_date_by_fighter.get(red_fighter, None)
    if red_last_date is None:
        red_days_since_last_fight.append(np.nan)
    else:
        red_days_since_last_fight.append((current_date - red_last_date).days)

    # Days since last fight for Blue corner
    blue_last_date = last_fight_date_by_fighter.get(blue_fighter, None)
    if blue_last_date is None:
        blue_days_since_last_fight.append(np.nan)
    else:
        blue_days_since_last_fight.append((current_date - blue_last_date).days)

    # Update last seen date for both fighters
    last_fight_date_by_fighter[red_fighter] = current_date
    last_fight_date_by_fighter[blue_fighter] = current_date

# Add the results as new columns in the dataframe
ufc_raw['RedDaysSinceLastFight'] = red_days_since_last_fight
ufc_raw['BlueDaysSinceLastFight'] = blue_days_since_last_fight

In [38]:
ufc_raw[['RedFighter', 'Date','RedDaysSinceLastFight']].sample(5)

,RedFighter,Date,RedDaysSinceLastFight
4129,Karolina Kowalkiewicz,2020-02-22,259.0
4777,Carlos Condit,2021-07-10,175.0
4221,Aljamain Sterling,2020-06-06,364.0
3369,Alex Perez,2018-08-04,161.0
6000,Rob Font,2023-12-02,119.0


In [39]:
ufc_raw[ufc_raw['RedFighter'] == 'Anderson Silva'][['RedFighter', 'Date', 'RedDaysSinceLastFight']]

,RedFighter,Date,RedDaysSinceLastFight
39,Anderson Silva,2010-04-10,NaN
109,Anderson Silva,2010-08-07,119.0
238,Anderson Silva,2011-02-05,182.0
386,Anderson Silva,2011-08-27,203.0
689,Anderson Silva,2012-07-07,315.0
773,Anderson Silva,2012-10-13,98.0
1019,Anderson Silva,2013-07-06,266.0
2245,Anderson Silva,2016-02-27,791.0
2696,Anderson Silva,2017-02-11,217.0


## Create DaysSinceDebut column: Days since his first UFC fight

In [40]:
# Step 1: Build debut date dictionary for all fighters
debut_date_by_fighter = {}

# Iterate through the dataset chronologically
for idx, row in ufc_raw.iterrows():
    current_date = row['Date']
    red_fighter = row['RedFighter']
    blue_fighter = row['BlueFighter']

    # Store the first appearance date if not already present
    if red_fighter not in debut_date_by_fighter:
        debut_date_by_fighter[red_fighter] = current_date
    if blue_fighter not in debut_date_by_fighter:
        debut_date_by_fighter[blue_fighter] = current_date

# Step 2: Compute days since debut for Red and Blue corners
ufc_raw['RedDaysSinceDebut'] = ufc_raw.apply(
    lambda row: (row['Date'] - debut_date_by_fighter[row['RedFighter']]).days,
    axis=1
)

ufc_raw['BlueDaysSinceDebut'] = ufc_raw.apply(
    lambda row: (row['Date'] - debut_date_by_fighter[row['BlueFighter']]).days,
    axis=1
)

In [41]:
ufc_raw[ufc_raw['RedFighter'] == 'Anderson Silva'][['RedFighter', 'Date', 'RedDaysSinceLastFight', 'RedDaysSinceDebut']]

,RedFighter,Date,RedDaysSinceLastFight,RedDaysSinceDebut
39,Anderson Silva,2010-04-10,NaN,0
109,Anderson Silva,2010-08-07,119.0,119
238,Anderson Silva,2011-02-05,182.0,301
386,Anderson Silva,2011-08-27,203.0,504
689,Anderson Silva,2012-07-07,315.0,819
773,Anderson Silva,2012-10-13,98.0,917
1019,Anderson Silva,2013-07-06,266.0,1183
2245,Anderson Silva,2016-02-27,791.0,2149
2696,Anderson Silva,2017-02-11,217.0,2499


In [42]:
# Step 1: Create debut flags
ufc_raw['RedIsDebut'] = ufc_raw['RedDaysSinceLastFight'].isna().map({True: 'Yes', False: 'No'})
ufc_raw['BlueIsDebut'] = ufc_raw['BlueDaysSinceLastFight'].isna().map({True: 'Yes', False: 'No'})

# Step 2: Impute NaN with median value in the debut.
ufc_raw['RedDaysSinceLastFight'].fillna(180, inplace=True) 
ufc_raw['BlueDaysSinceLastFight'].fillna(180, inplace=True)

/tmp/ipykernel_13127/1337356144.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ufc_raw['RedDaysSinceLastFight'].fillna(180, inplace=True)
/tmp/ipykernel_13127/1337356144.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [43]:
ufc_raw[ufc_raw['RedFighter'] == 'Anderson Silva'][['RedFighter', 'Date', 'RedDaysSinceLastFight', 'RedDaysSinceDebut', 'RedIsDebut']]

,RedFighter,Date,RedDaysSinceLastFight,RedDaysSinceDebut,RedIsDebut
39,Anderson Silva,2010-04-10,180.0,0,Yes
109,Anderson Silva,2010-08-07,119.0,119,No
238,Anderson Silva,2011-02-05,182.0,301,No
386,Anderson Silva,2011-08-27,203.0,504,No
689,Anderson Silva,2012-07-07,315.0,819,No
773,Anderson Silva,2012-10-13,98.0,917,No
1019,Anderson Silva,2013-07-06,266.0,1183,No
2245,Anderson Silva,2016-02-27,791.0,2149,No
2696,Anderson Silva,2017-02-11,217.0,2499,No


## Create FightsInLastYear/FightsInLast6Months : Number of fights in the last year/6 months

In [44]:
from collections import defaultdict

# Dictionary of past fight dates for each fighter
fight_history = defaultdict(list)

# Output lists
red_fights_last_year = []
blue_fights_last_year = []

# Iterate over each row in chronological order
for idx, row in ufc_raw.iterrows():
    current_date = row['Date']
    red_fighter = row['RedFighter']
    blue_fighter = row['BlueFighter']

    # Count red's fights in the past 365 days
    red_past_fights = [d for d in fight_history[red_fighter] if 0 < (current_date - d).days <= 365]
    red_fights_last_year.append(len(red_past_fights))

    # Count blue's fights in the past 365 days
    blue_past_fights = [d for d in fight_history[blue_fighter] if 0 < (current_date - d).days <= 365]
    blue_fights_last_year.append(len(blue_past_fights))

    # Update fight history with current fight date
    fight_history[red_fighter].append(current_date)
    fight_history[blue_fighter].append(current_date)

# Add the new columns to the dataframe
ufc_raw['RedFightsInLastYear'] = red_fights_last_year
ufc_raw['BlueFightsInLastYear'] = blue_fights_last_year

In [45]:
from collections import defaultdict

# Dictionary to store past fight dates for each fighter
fight_history_6m = defaultdict(list)

# Output columns
red_fights_last_6m = []
blue_fights_last_6m = []

# Iterate through the dataset chronologically
for idx, row in ufc_raw.iterrows():
    current_date = row['Date']
    red_fighter = row['RedFighter']
    blue_fighter = row['BlueFighter']

    # Count red's fights in the past 183 days (6 months)
    red_past_fights = [d for d in fight_history_6m[red_fighter] if 0 < (current_date - d).days <= 183]
    red_fights_last_6m.append(len(red_past_fights))

    # Count blue's fights in the past 183 days (6 months)
    blue_past_fights = [d for d in fight_history_6m[blue_fighter] if 0 < (current_date - d).days <= 183]
    blue_fights_last_6m.append(len(blue_past_fights))

    # Update fight history with current fight
    fight_history_6m[red_fighter].append(current_date)
    fight_history_6m[blue_fighter].append(current_date)

# Assign to new columns
ufc_raw['RedFightsInLast6Months'] = red_fights_last_6m
ufc_raw['BlueFightsInLast6Months'] = blue_fights_last_6m

In [46]:
ufc_raw[ufc_raw['RedFighter'] == 'Anderson Silva'][['RedFighter', 'Date', 'RedDaysSinceLastFight', 'RedDaysSinceDebut', 'RedIsDebut', 'RedFightsInLastYear', 'RedFightsInLast6Months']]

,RedFighter,Date,RedDaysSinceLastFight,RedDaysSinceDebut,RedIsDebut,RedFightsInLastYear,RedFightsInLast6Months
39,Anderson Silva,2010-04-10,180.0,0,Yes,0,0
109,Anderson Silva,2010-08-07,119.0,119,No,1,1
238,Anderson Silva,2011-02-05,182.0,301,No,2,1
386,Anderson Silva,2011-08-27,203.0,504,No,1,0
689,Anderson Silva,2012-07-07,315.0,819,No,1,0
773,Anderson Silva,2012-10-13,98.0,917,No,1,1
1019,Anderson Silva,2013-07-06,266.0,1183,No,2,0
2245,Anderson Silva,2016-02-27,791.0,2149,No,0,0
2696,Anderson Silva,2017-02-11,217.0,2499,No,2,0


## Create DaysSinceLastWin : Number of fights in the last year/6 months

In [47]:
from collections import defaultdict

# Initialize win history tracker
last_win_date_by_fighter = {}

# Output columns
red_days_since_last_win = []
blue_days_since_last_win = []

# Iterate over ufc_raw in chronological order
for idx, row in ufc_raw.iterrows():
    date = row['Date']
    red = row['RedFighter']
    blue = row['BlueFighter']
    winner = row['Winner']  # Should be 'Red', 'Blue' or 'Draw'

    # Red corner
    red_last_win_date = last_win_date_by_fighter.get(red, None)
    if red_last_win_date is None:
        red_days_since_last_win.append(np.nan)
    else:
        red_days_since_last_win.append((date - red_last_win_date).days)

    # Blue corner
    blue_last_win_date = last_win_date_by_fighter.get(blue, None)
    if blue_last_win_date is None:
        blue_days_since_last_win.append(np.nan)
    else:
        blue_days_since_last_win.append((date - blue_last_win_date).days)

    # Update win dates
    if winner == 'Red':
        last_win_date_by_fighter[red] = date
    elif winner == 'Blue':
        last_win_date_by_fighter[blue] = date
    # No update for draw

# Add to DataFrame
ufc_raw['RedDaysSinceLastWin'] = red_days_since_last_win
ufc_raw['BlueDaysSinceLastWin'] = blue_days_since_last_win

In [48]:
ufc_raw[ufc_raw['RedFighter'] == 'Anderson Silva'][['RedFighter', 'Date', 'RedDaysSinceLastFight', 'RedDaysSinceDebut', 'RedIsDebut', 'RedFightsInLastYear', 'RedFightsInLast6Months', 'RedDaysSinceLastWin']]

,RedFighter,Date,RedDaysSinceLastFight,RedDaysSinceDebut,RedIsDebut,RedFightsInLastYear,RedFightsInLast6Months,RedDaysSinceLastWin
39,Anderson Silva,2010-04-10,180.0,0,Yes,0,0,NaN
109,Anderson Silva,2010-08-07,119.0,119,No,1,1,119.0
238,Anderson Silva,2011-02-05,182.0,301,No,2,1,182.0
386,Anderson Silva,2011-08-27,203.0,504,No,1,0,203.0
689,Anderson Silva,2012-07-07,315.0,819,No,1,0,315.0
773,Anderson Silva,2012-10-13,98.0,917,No,1,1,98.0
1019,Anderson Silva,2013-07-06,266.0,1183,No,2,0,266.0
2245,Anderson Silva,2016-02-27,791.0,2149,No,0,0,1232.0
2696,Anderson Silva,2017-02-11,217.0,2499,No,2,0,1582.0


In [49]:
ufc_raw['RedDaysSinceLastWin'].median()

246.0

In [50]:
# FIll with median, moodel already know if it is the first fight of a fighter
ufc_raw['RedDaysSinceLastWin'].fillna(246, inplace=True)
ufc_raw['BlueDaysSinceLastWin'].fillna(246, inplace=True)

/tmp/ipykernel_13127/3507902234.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ufc_raw['RedDaysSinceLastWin'].fillna(246, inplace=True)
/tmp/ipykernel_13127/3507902234.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [51]:
ufc_raw[ufc_raw['RedFighter'] == 'Anderson Silva'][['RedFighter', 'Date', 'RedDaysSinceLastFight', 'RedDaysSinceDebut', 'RedIsDebut', 'RedFightsInLastYear', 'RedFightsInLast6Months', 'RedDaysSinceLastWin']]

,RedFighter,Date,RedDaysSinceLastFight,RedDaysSinceDebut,RedIsDebut,RedFightsInLastYear,RedFightsInLast6Months,RedDaysSinceLastWin
39,Anderson Silva,2010-04-10,180.0,0,Yes,0,0,246.0
109,Anderson Silva,2010-08-07,119.0,119,No,1,1,119.0
238,Anderson Silva,2011-02-05,182.0,301,No,2,1,182.0
386,Anderson Silva,2011-08-27,203.0,504,No,1,0,203.0
689,Anderson Silva,2012-07-07,315.0,819,No,1,0,315.0
773,Anderson Silva,2012-10-13,98.0,917,No,1,1,98.0
1019,Anderson Silva,2013-07-06,266.0,1183,No,2,0,266.0
2245,Anderson Silva,2016-02-27,791.0,2149,No,0,0,1232.0
2696,Anderson Silva,2017-02-11,217.0,2499,No,2,0,1582.0


In [52]:
# Save the cleaned file
ufc_raw.to_csv(f'{project_root}/data/raw/ufc_raw_dates_ft.csv', index=False)
logger.info("✅ Raw with Date Features file saved as 'ufc_raw_dates_ft.csv'.")

[INFO] ✅ Raw with Date Features file saved as 'ufc_raw_dates_ft.csv'.


<div style="text-align: center;">
     <img src="../img/ufc_logo.png" width="800" /> 
</div>